In [2]:
import arcpy

# 1. Preparing the Data

In [3]:
data1 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Admin_Level.shp" #Administrative Boundaries
data2 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Monitor_Stations.shp" #Monitor Stations
data3 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\n36_e116_1arc_v3.tif" #DEM 1
data4 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\n36_e117_1arc_v3.tif" #DEM 2
data5 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Nature_L_Jinan.shp" #Natural Elements Line
data6 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Nature_P_Jinan.shp" #Natural Elements Polygon
data7 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Water_L_Jinan.shp" #Water Bodies Line
data8 = r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Water_P_Jinan.shp" #Water Bodies Polygon

In [4]:
project = arcpy.mp.ArcGISProject("CURRENT")

map = project.activeMap
map.addDataFromPath(data1)
map.addDataFromPath(data2)
map.addDataFromPath(data3)
map.addDataFromPath(data4)
map.addDataFromPath(data5)
map.addDataFromPath(data6)
map.addDataFromPath(data7)
map.addDataFromPath(data8)

# 2. Generate Raster Table of Flooding Risk

## Merge DEM

In [5]:
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb", workspace=r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb"):
    arcpy.management.MosaicToNewRaster("n36_e117_1arc_v3.tif;n36_e116_1arc_v3.tif", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb", "Merged_DEM.tif", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

## IDW Interpolation and Raster Calculation

In [11]:
arcpy.ddd.Idw("Monitor_Stations", "Warning", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\IDW", 0.000381676048000145, 2, "VARIABLE 22", None)

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\IDW'>

In [16]:
output_raster = arcpy.ia.RasterCalculator(["IDW", "tif"],
                                       ["x", "y"], "x - y"); output_raster.save(r"c:\Users\zhaoxu sui\documents\GitHub\GIS5571\final project\final_project\final_project_5571\final_project_5571.gdb\IDW_Flooding_Risk")

## Kriging Interpolation and Raster Calculation

In [17]:
arcpy.ddd.Kriging("Monitor_Stations", "Warning", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Kriging", "Spherical # # # #", 0.000381676048000145, "VARIABLE 22", None)

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Kriging'>

In [18]:
output_raster = arcpy.ia.RasterCalculator(["Kriging", "tif"],
                                       ["x", "y"], "x - y"); output_raster.save(r"c:\Users\zhaoxu sui\documents\GitHub\GIS5571\final project\final_project\final_project_5571\final_project_5571.gdb\Kriging_Flooding_Risk")

# 3. Clip Rasters

## Make Buffers

In [19]:
#Water Line Buffers
arcpy.analysis.Buffer("Water_L_Jinan", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Water_L_Buffer", "100 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Water_L_Buffer'>

In [20]:
#Water Polygon Buffers
arcpy.management.SelectLayerByAttribute("Nature_P_Jinan", "NEW_SELECTION", "natural = 'water'", None)

<Result 'Nature_P_Jinan'>

In [21]:
arcpy.analysis.Buffer("Nature_P_Jinan", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Water_P_Buffer", "100 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Water_P_Buffer'>

In [22]:
#Merge River Buffers and Polygon Buffers and Dissolve the feature
arcpy.management.Merge("Water_P_Buffer;Water_L_Buffer", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Buffer_Merge", 'Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Water_P_Buffer,Shape_Length,-1,-1,Water_L_Buffer,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,Water_P_Buffer,Shape_Area,-1,-1,Water_L_Buffer,Shape_Area,-1,-1', "NO_SOURCE_INFO")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Buffer_Merge'>

In [23]:
arcpy.management.Dissolve("Buffer_Merge", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Buffer", None, None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Buffer'>

## Clip Rasters to locate all the risky area

In [25]:
IDW_Extracted = arcpy.sa.ExtractByMask("IDW_Flooding_Risk", "Buffer"); out_raster.save(r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\IDW_Extracted")

In [26]:
Kriging_Extracted = arcpy.sa.ExtractByMask("Kriging_Flooding_Risk", "Buffer"); out_raster.save(r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Kriging_Extracted")

# 4. Integrate into Communities

In [27]:
#Rasters to Points
arcpy.conversion.RasterToPoint("IDW_Extracted", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\IDW_Points", "Value")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\IDW_Points'>

In [28]:
arcpy.conversion.RasterToPoint("Kriging_Extracted", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Kriging_Points", "Value")

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Kriging_Points'>

In [34]:
#Spatial Joins
arcpy.analysis.SpatialJoin("Admin_Level", "IDW_Points", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Admin_IDW", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'full_id "full_id" true true false 254 Text 0 0,First,#,Admin_Level,full_id,0,254;osm_id "osm_id" true true false 254 Text 0 0,First,#,Admin_Level,osm_id,0,254;osm_type "osm_type" true true false 254 Text 0 0,First,#,Admin_Level,osm_type,0,254;admin_leve "admin_leve" true true false 254 Text 0 0,First,#,Admin_Level,admin_leve,0,254;old_name_z "old_name_z" true true false 254 Text 0 0,First,#,Admin_Level,old_name_z,0,254;old_name_e "old_name_e" true true false 254 Text 0 0,First,#,Admin_Level,old_name_e,0,254;old_name "old_name" true true false 254 Text 0 0,First,#,Admin_Level,old_name,0,254;alt_name_z "alt_name_z" true true false 254 Text 0 0,First,#,Admin_Level,alt_name_z,0,254;alt_name_e "alt_name_e" true true false 254 Text 0 0,First,#,Admin_Level,alt_name_e,0,254;postal_cod "postal_cod" true true false 254 Text 0 0,First,#,Admin_Level,postal_cod,0,254;division_c "division_c" true true false 254 Text 0 0,First,#,Admin_Level,division_c,0,254;source_nam "source_nam" true true false 254 Text 0 0,First,#,Admin_Level,source_nam,0,254;wikipedia "wikipedia" true true false 254 Text 0 0,First,#,Admin_Level,wikipedia,0,254;wikidata "wikidata" true true false 254 Text 0 0,First,#,Admin_Level,wikidata,0,254;type "type" true true false 254 Text 0 0,First,#,Admin_Level,type,0,254;timezone "timezone" true true false 254 Text 0 0,First,#,Admin_Level,timezone,0,254;short_name "short_name" true true false 254 Text 0 0,First,#,Admin_Level,short_name,0,254;ref "ref" true true false 254 Text 0 0,First,#,Admin_Level,ref,0,254;official_n "official_n" true true false 254 Text 0 0,First,#,Admin_Level,official_n,0,254;name_zh_pi "name_zh_pi" true true false 254 Text 0 0,First,#,Admin_Level,name_zh_pi,0,254;name_zh_Ha "name_zh-Ha" true true false 254 Text 0 0,First,#,Admin_Level,name_zh-Ha,0,254;name_zh__1 "name_zh-_1" true true false 254 Text 0 0,First,#,Admin_Level,name_zh-_1,0,254;name_zh "name_zh" true true false 254 Text 0 0,First,#,Admin_Level,name_zh,0,254;name_za "name_za" true true false 254 Text 0 0,First,#,Admin_Level,name_za,0,254;name_yue "name_yue" true true false 254 Text 0 0,First,#,Admin_Level,name_yue,0,254;name_wa "name_wa" true true false 254 Text 0 0,First,#,Admin_Level,name_wa,0,254;name_vi "name_vi" true true false 254 Text 0 0,First,#,Admin_Level,name_vi,0,254;name_uz "name_uz" true true false 254 Text 0 0,First,#,Admin_Level,name_uz,0,254;name_ur "name_ur" true true false 254 Text 0 0,First,#,Admin_Level,name_ur,0,254;name_uk "name_uk" true true false 254 Text 0 0,First,#,Admin_Level,name_uk,0,254;name_ug "name_ug" true true false 254 Text 0 0,First,#,Admin_Level,name_ug,0,254;name_tr "name_tr" true true false 254 Text 0 0,First,#,Admin_Level,name_tr,0,254;name_tl "name_tl" true true false 254 Text 0 0,First,#,Admin_Level,name_tl,0,254;name_th "name_th" true true false 254 Text 0 0,First,#,Admin_Level,name_th,0,254;name_tg "name_tg" true true false 254 Text 0 0,First,#,Admin_Level,name_tg,0,254;name_ta "name_ta" true true false 254 Text 0 0,First,#,Admin_Level,name_ta,0,254;name_sw "name_sw" true true false 254 Text 0 0,First,#,Admin_Level,name_sw,0,254;name_sv "name_sv" true true false 254 Text 0 0,First,#,Admin_Level,name_sv,0,254;name_sr "name_sr" true true false 254 Text 0 0,First,#,Admin_Level,name_sr,0,254;name_ru "name_ru" true true false 254 Text 0 0,First,#,Admin_Level,name_ru,0,254;name_ro "name_ro" true true false 254 Text 0 0,First,#,Admin_Level,name_ro,0,254;name_qu "name_qu" true true false 254 Text 0 0,First,#,Admin_Level,name_qu,0,254;name_pt "name_pt" true true false 254 Text 0 0,First,#,Admin_Level,name_pt,0,254;name_pl "name_pl" true true false 254 Text 0 0,First,#,Admin_Level,name_pl,0,254;name_os "name_os" true true false 254 Text 0 0,First,#,Admin_Level,name_os,0,254;name_no "name_no" true true false 254 Text 0 0,First,#,Admin_Level,name_no,0,254;name_nn "name_nn" true true false 254 Text 0 0,First,#,Admin_Level,name_nn,0,254;name_nl "name_nl" true true false 254 Text 0 0,First,#,Admin_Level,name_nl,0,254;name_nan "name_nan" true true false 254 Text 0 0,First,#,Admin_Level,name_nan,0,254;name_ms "name_ms" true true false 254 Text 0 0,First,#,Admin_Level,name_ms,0,254;name_mr "name_mr" true true false 254 Text 0 0,First,#,Admin_Level,name_mr,0,254;name_mn "name_mn" true true false 254 Text 0 0,First,#,Admin_Level,name_mn,0,254;name_ml "name_ml" true true false 254 Text 0 0,First,#,Admin_Level,name_ml,0,254;name_mg "name_mg" true true false 254 Text 0 0,First,#,Admin_Level,name_mg,0,254;name_lzh "name_lzh" true true false 254 Text 0 0,First,#,Admin_Level,name_lzh,0,254;name_lv "name_lv" true true false 254 Text 0 0,First,#,Admin_Level,name_lv,0,254;name_lt "name_lt" true true false 254 Text 0 0,First,#,Admin_Level,name_lt,0,254;name_la "name_la" true true false 254 Text 0 0,First,#,Admin_Level,name_la,0,254;name_ko "name_ko" true true false 254 Text 0 0,First,#,Admin_Level,name_ko,0,254;name_kg "name_kg" true true false 254 Text 0 0,First,#,Admin_Level,name_kg,0,254;name_ka "name_ka" true true false 254 Text 0 0,First,#,Admin_Level,name_ka,0,254;name_ja "name_ja" true true false 254 Text 0 0,First,#,Admin_Level,name_ja,0,254;name_it "name_it" true true false 254 Text 0 0,First,#,Admin_Level,name_it,0,254;name_id "name_id" true true false 254 Text 0 0,First,#,Admin_Level,name_id,0,254;name_ia "name_ia" true true false 254 Text 0 0,First,#,Admin_Level,name_ia,0,254;name_hy "name_hy" true true false 254 Text 0 0,First,#,Admin_Level,name_hy,0,254;name_hu "name_hu" true true false 254 Text 0 0,First,#,Admin_Level,name_hu,0,254;name_hr "name_hr" true true false 254 Text 0 0,First,#,Admin_Level,name_hr,0,254;name_hi "name_hi" true true false 254 Text 0 0,First,#,Admin_Level,name_hi,0,254;name_he "name_he" true true false 254 Text 0 0,First,#,Admin_Level,name_he,0,254;name_gv "name_gv" true true false 254 Text 0 0,First,#,Admin_Level,name_gv,0,254;name_gl "name_gl" true true false 254 Text 0 0,First,#,Admin_Level,name_gl,0,254;name_ga "name_ga" true true false 254 Text 0 0,First,#,Admin_Level,name_ga,0,254;name_fr "name_fr" true true false 254 Text 0 0,First,#,Admin_Level,name_fr,0,254;name_fi "name_fi" true true false 254 Text 0 0,First,#,Admin_Level,name_fi,0,254;name_fa "name_fa" true true false 254 Text 0 0,First,#,Admin_Level,name_fa,0,254;name_eu "name_eu" true true false 254 Text 0 0,First,#,Admin_Level,name_eu,0,254;name_et "name_et" true true false 254 Text 0 0,First,#,Admin_Level,name_et,0,254;name_es "name_es" true true false 254 Text 0 0,First,#,Admin_Level,name_es,0,254;name_eo "name_eo" true true false 254 Text 0 0,First,#,Admin_Level,name_eo,0,254;name_en "name_en" true true false 254 Text 0 0,First,#,Admin_Level,name_en,0,254;name_el "name_el" true true false 254 Text 0 0,First,#,Admin_Level,name_el,0,254;name_de "name_de" true true false 254 Text 0 0,First,#,Admin_Level,name_de,0,254;name_da "name_da" true true false 254 Text 0 0,First,#,Admin_Level,name_da,0,254;name_cy "name_cy" true true false 254 Text 0 0,First,#,Admin_Level,name_cy,0,254;name_cs "name_cs" true true false 254 Text 0 0,First,#,Admin_Level,name_cs,0,254;name_ce "name_ce" true true false 254 Text 0 0,First,#,Admin_Level,name_ce,0,254;name_ca "name_ca" true true false 254 Text 0 0,First,#,Admin_Level,name_ca,0,254;name_br "name_br" true true false 254 Text 0 0,First,#,Admin_Level,name_br,0,254;name_bo "name_bo" true true false 254 Text 0 0,First,#,Admin_Level,name_bo,0,254;name_bn "name_bn" true true false 254 Text 0 0,First,#,Admin_Level,name_bn,0,254;name_bg "name_bg" true true false 254 Text 0 0,First,#,Admin_Level,name_bg,0,254;name_be "name_be" true true false 254 Text 0 0,First,#,Admin_Level,name_be,0,254;name_ar "name_ar" true true false 254 Text 0 0,First,#,Admin_Level,name_ar,0,254;name_af "name_af" true true false 254 Text 0 0,First,#,Admin_Level,name_af,0,254;name "name" true true false 254 Text 0 0,First,#,Admin_Level,name,0,254;is_in_coun "is_in_coun" true true false 254 Text 0 0,First,#,Admin_Level,is_in_coun,0,254;is_in_co_1 "is_in_co_1" true true false 254 Text 0 0,First,#,Admin_Level,is_in_co_1,0,254;is_in_cont "is_in_cont" true true false 254 Text 0 0,First,#,Admin_Level,is_in_cont,0,254;int_name "int_name" true true false 254 Text 0 0,First,#,Admin_Level,int_name,0,254;boundary "boundary" true true false 254 Text 0 0,First,#,Admin_Level,boundary,0,254;alt_name "alt_name" true true false 254 Text 0 0,First,#,Admin_Level,alt_name,0,254;ISO3166_2 "ISO3166-2" true true false 254 Text 0 0,First,#,Admin_Level,ISO3166-2,0,254;pointid "pointid" true true false 4 Long 0 0,First,#,IDW_Points,pointid,-1,-1;grid_code "grid_code" true true false 8 Double 0 0,Sum,#,IDW_Points,grid_code,-1,-1', "INTERSECT", None, '')

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Admin_IDW'>

In [35]:
arcpy.analysis.SpatialJoin("Admin_Level", "Kriging_Points", r"C:\Users\Zhaoxu Sui\Documents\GitHub\GIS5571\Final Project\Final_Project\Final_Project_5571\Final_Project_5571.gdb\Admin_Kriging", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'full_id "full_id" true true false 254 Text 0 0,First,#,Admin_Level,full_id,0,254;osm_id "osm_id" true true false 254 Text 0 0,First,#,Admin_Level,osm_id,0,254;osm_type "osm_type" true true false 254 Text 0 0,First,#,Admin_Level,osm_type,0,254;admin_leve "admin_leve" true true false 254 Text 0 0,First,#,Admin_Level,admin_leve,0,254;old_name_z "old_name_z" true true false 254 Text 0 0,First,#,Admin_Level,old_name_z,0,254;old_name_e "old_name_e" true true false 254 Text 0 0,First,#,Admin_Level,old_name_e,0,254;old_name "old_name" true true false 254 Text 0 0,First,#,Admin_Level,old_name,0,254;alt_name_z "alt_name_z" true true false 254 Text 0 0,First,#,Admin_Level,alt_name_z,0,254;alt_name_e "alt_name_e" true true false 254 Text 0 0,First,#,Admin_Level,alt_name_e,0,254;postal_cod "postal_cod" true true false 254 Text 0 0,First,#,Admin_Level,postal_cod,0,254;division_c "division_c" true true false 254 Text 0 0,First,#,Admin_Level,division_c,0,254;source_nam "source_nam" true true false 254 Text 0 0,First,#,Admin_Level,source_nam,0,254;wikipedia "wikipedia" true true false 254 Text 0 0,First,#,Admin_Level,wikipedia,0,254;wikidata "wikidata" true true false 254 Text 0 0,First,#,Admin_Level,wikidata,0,254;type "type" true true false 254 Text 0 0,First,#,Admin_Level,type,0,254;timezone "timezone" true true false 254 Text 0 0,First,#,Admin_Level,timezone,0,254;short_name "short_name" true true false 254 Text 0 0,First,#,Admin_Level,short_name,0,254;ref "ref" true true false 254 Text 0 0,First,#,Admin_Level,ref,0,254;official_n "official_n" true true false 254 Text 0 0,First,#,Admin_Level,official_n,0,254;name_zh_pi "name_zh_pi" true true false 254 Text 0 0,First,#,Admin_Level,name_zh_pi,0,254;name_zh_Ha "name_zh-Ha" true true false 254 Text 0 0,First,#,Admin_Level,name_zh-Ha,0,254;name_zh__1 "name_zh-_1" true true false 254 Text 0 0,First,#,Admin_Level,name_zh-_1,0,254;name_zh "name_zh" true true false 254 Text 0 0,First,#,Admin_Level,name_zh,0,254;name_za "name_za" true true false 254 Text 0 0,First,#,Admin_Level,name_za,0,254;name_yue "name_yue" true true false 254 Text 0 0,First,#,Admin_Level,name_yue,0,254;name_wa "name_wa" true true false 254 Text 0 0,First,#,Admin_Level,name_wa,0,254;name_vi "name_vi" true true false 254 Text 0 0,First,#,Admin_Level,name_vi,0,254;name_uz "name_uz" true true false 254 Text 0 0,First,#,Admin_Level,name_uz,0,254;name_ur "name_ur" true true false 254 Text 0 0,First,#,Admin_Level,name_ur,0,254;name_uk "name_uk" true true false 254 Text 0 0,First,#,Admin_Level,name_uk,0,254;name_ug "name_ug" true true false 254 Text 0 0,First,#,Admin_Level,name_ug,0,254;name_tr "name_tr" true true false 254 Text 0 0,First,#,Admin_Level,name_tr,0,254;name_tl "name_tl" true true false 254 Text 0 0,First,#,Admin_Level,name_tl,0,254;name_th "name_th" true true false 254 Text 0 0,First,#,Admin_Level,name_th,0,254;name_tg "name_tg" true true false 254 Text 0 0,First,#,Admin_Level,name_tg,0,254;name_ta "name_ta" true true false 254 Text 0 0,First,#,Admin_Level,name_ta,0,254;name_sw "name_sw" true true false 254 Text 0 0,First,#,Admin_Level,name_sw,0,254;name_sv "name_sv" true true false 254 Text 0 0,First,#,Admin_Level,name_sv,0,254;name_sr "name_sr" true true false 254 Text 0 0,First,#,Admin_Level,name_sr,0,254;name_ru "name_ru" true true false 254 Text 0 0,First,#,Admin_Level,name_ru,0,254;name_ro "name_ro" true true false 254 Text 0 0,First,#,Admin_Level,name_ro,0,254;name_qu "name_qu" true true false 254 Text 0 0,First,#,Admin_Level,name_qu,0,254;name_pt "name_pt" true true false 254 Text 0 0,First,#,Admin_Level,name_pt,0,254;name_pl "name_pl" true true false 254 Text 0 0,First,#,Admin_Level,name_pl,0,254;name_os "name_os" true true false 254 Text 0 0,First,#,Admin_Level,name_os,0,254;name_no "name_no" true true false 254 Text 0 0,First,#,Admin_Level,name_no,0,254;name_nn "name_nn" true true false 254 Text 0 0,First,#,Admin_Level,name_nn,0,254;name_nl "name_nl" true true false 254 Text 0 0,First,#,Admin_Level,name_nl,0,254;name_nan "name_nan" true true false 254 Text 0 0,First,#,Admin_Level,name_nan,0,254;name_ms "name_ms" true true false 254 Text 0 0,First,#,Admin_Level,name_ms,0,254;name_mr "name_mr" true true false 254 Text 0 0,First,#,Admin_Level,name_mr,0,254;name_mn "name_mn" true true false 254 Text 0 0,First,#,Admin_Level,name_mn,0,254;name_ml "name_ml" true true false 254 Text 0 0,First,#,Admin_Level,name_ml,0,254;name_mg "name_mg" true true false 254 Text 0 0,First,#,Admin_Level,name_mg,0,254;name_lzh "name_lzh" true true false 254 Text 0 0,First,#,Admin_Level,name_lzh,0,254;name_lv "name_lv" true true false 254 Text 0 0,First,#,Admin_Level,name_lv,0,254;name_lt "name_lt" true true false 254 Text 0 0,First,#,Admin_Level,name_lt,0,254;name_la "name_la" true true false 254 Text 0 0,First,#,Admin_Level,name_la,0,254;name_ko "name_ko" true true false 254 Text 0 0,First,#,Admin_Level,name_ko,0,254;name_kg "name_kg" true true false 254 Text 0 0,First,#,Admin_Level,name_kg,0,254;name_ka "name_ka" true true false 254 Text 0 0,First,#,Admin_Level,name_ka,0,254;name_ja "name_ja" true true false 254 Text 0 0,First,#,Admin_Level,name_ja,0,254;name_it "name_it" true true false 254 Text 0 0,First,#,Admin_Level,name_it,0,254;name_id "name_id" true true false 254 Text 0 0,First,#,Admin_Level,name_id,0,254;name_ia "name_ia" true true false 254 Text 0 0,First,#,Admin_Level,name_ia,0,254;name_hy "name_hy" true true false 254 Text 0 0,First,#,Admin_Level,name_hy,0,254;name_hu "name_hu" true true false 254 Text 0 0,First,#,Admin_Level,name_hu,0,254;name_hr "name_hr" true true false 254 Text 0 0,First,#,Admin_Level,name_hr,0,254;name_hi "name_hi" true true false 254 Text 0 0,First,#,Admin_Level,name_hi,0,254;name_he "name_he" true true false 254 Text 0 0,First,#,Admin_Level,name_he,0,254;name_gv "name_gv" true true false 254 Text 0 0,First,#,Admin_Level,name_gv,0,254;name_gl "name_gl" true true false 254 Text 0 0,First,#,Admin_Level,name_gl,0,254;name_ga "name_ga" true true false 254 Text 0 0,First,#,Admin_Level,name_ga,0,254;name_fr "name_fr" true true false 254 Text 0 0,First,#,Admin_Level,name_fr,0,254;name_fi "name_fi" true true false 254 Text 0 0,First,#,Admin_Level,name_fi,0,254;name_fa "name_fa" true true false 254 Text 0 0,First,#,Admin_Level,name_fa,0,254;name_eu "name_eu" true true false 254 Text 0 0,First,#,Admin_Level,name_eu,0,254;name_et "name_et" true true false 254 Text 0 0,First,#,Admin_Level,name_et,0,254;name_es "name_es" true true false 254 Text 0 0,First,#,Admin_Level,name_es,0,254;name_eo "name_eo" true true false 254 Text 0 0,First,#,Admin_Level,name_eo,0,254;name_en "name_en" true true false 254 Text 0 0,First,#,Admin_Level,name_en,0,254;name_el "name_el" true true false 254 Text 0 0,First,#,Admin_Level,name_el,0,254;name_de "name_de" true true false 254 Text 0 0,First,#,Admin_Level,name_de,0,254;name_da "name_da" true true false 254 Text 0 0,First,#,Admin_Level,name_da,0,254;name_cy "name_cy" true true false 254 Text 0 0,First,#,Admin_Level,name_cy,0,254;name_cs "name_cs" true true false 254 Text 0 0,First,#,Admin_Level,name_cs,0,254;name_ce "name_ce" true true false 254 Text 0 0,First,#,Admin_Level,name_ce,0,254;name_ca "name_ca" true true false 254 Text 0 0,First,#,Admin_Level,name_ca,0,254;name_br "name_br" true true false 254 Text 0 0,First,#,Admin_Level,name_br,0,254;name_bo "name_bo" true true false 254 Text 0 0,First,#,Admin_Level,name_bo,0,254;name_bn "name_bn" true true false 254 Text 0 0,First,#,Admin_Level,name_bn,0,254;name_bg "name_bg" true true false 254 Text 0 0,First,#,Admin_Level,name_bg,0,254;name_be "name_be" true true false 254 Text 0 0,First,#,Admin_Level,name_be,0,254;name_ar "name_ar" true true false 254 Text 0 0,First,#,Admin_Level,name_ar,0,254;name_af "name_af" true true false 254 Text 0 0,First,#,Admin_Level,name_af,0,254;name "name" true true false 254 Text 0 0,First,#,Admin_Level,name,0,254;is_in_coun "is_in_coun" true true false 254 Text 0 0,First,#,Admin_Level,is_in_coun,0,254;is_in_co_1 "is_in_co_1" true true false 254 Text 0 0,First,#,Admin_Level,is_in_co_1,0,254;is_in_cont "is_in_cont" true true false 254 Text 0 0,First,#,Admin_Level,is_in_cont,0,254;int_name "int_name" true true false 254 Text 0 0,First,#,Admin_Level,int_name,0,254;boundary "boundary" true true false 254 Text 0 0,First,#,Admin_Level,boundary,0,254;alt_name "alt_name" true true false 254 Text 0 0,First,#,Admin_Level,alt_name,0,254;ISO3166_2 "ISO3166-2" true true false 254 Text 0 0,First,#,Admin_Level,ISO3166-2,0,254;pointid "pointid" true true false 4 Long 0 0,First,#,Kriging_Points,pointid,-1,-1;grid_code "grid_code" true true false 8 Double 0 0,Sum,#,Kriging_Points,grid_code,-1,-1', "INTERSECT", None, '')

<Result 'C:\\Users\\Zhaoxu Sui\\Documents\\GitHub\\GIS5571\\Final Project\\Final_Project\\Final_Project_5571\\Final_Project_5571.gdb\\Admin_Kriging'>